In [480]:
# SUUMOの物件情報を自動取得（スクレイピング）したのでコードを解説する。 https://qiita.com/tomyu/items/a08d3180b7cbe63667c9
# SUUMOのデータを前処理したので解説する。 https://qiita.com/tomyu/items/e039bcf2a44ad2e83b94
# SUUMOの前処理でもう少し手を加えたので解説する。 https://qiita.com/tomyu/items/ba376faa7a7939941eee

In [31]:
# ライブラリのインポート
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re  # 正規表現の使用

import gspread
from oauth2client.service_account import ServiceAccountCredentials

from dotenv import load_dotenv  # 環境変数の読み込みに使用。https://chat.openai.com/share/f6530203-f855-42b2-9488-089cc957dc63  # Pythonで.envファイルから環境変数を設定する https://qiita.com/wooooo/items/7b57eaf32c22195df843
import os  # Pythonの標準ライブラリの一部で、オペレーティングシステムに関連する機能を提供します。os.getenv はその中の一つの関数です。https://chat.openai.com/share/c3ef4fc3-51bf-4063-87f8-6c1cf65e5bfd

In [32]:
# 環境変数の読み込み
load_dotenv()

True

In [33]:
# 使用するGoogleSheetsAPI、GoogleDriveAPI情報の指定
SCOPES = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

# 秘密鍵のjsonファイルを指定
SEVICE_ACCOUNT_FILE = os.getenv('SEVICE_ACCOUNT_FILE')

# 認証情報を作成
credentials = ServiceAccountCredentials.from_json_keyfile_name(SEVICE_ACCOUNT_FILE, SCOPES)

# スプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

# 編集するスプレッドシートとワークシートの指定
SPREADSHEET_KEY = os.getenv('SPREADSHEET_KEY')
workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("シート1")

In [34]:
# 関数の定義

# 文字+数字+文字 → 文字
def objnumobj_num(x):
    return re.sub(r'\w+?([0-9]+)\w+',r'\1',x)
    # re.sub()は、第一引数に正規表現パターン、第二引数に置換先文字列、第三引数に処理対象の文字列を指定。https://note.nkmk.me/python-str-replace-translate-re-sub/#resub-resubn
    # r''は"\"を特殊シーケンスとして認識させるために必要なもの。
    # \wは、任意の英数字で、「1, 2, 333, 12aa, ab」などのこと。漢字にも使えている。☛ なぜ漢字は/Wではないのか？https://chat.openai.com/share/6f7b2fd9-26a1-49da-bf35-501d4649b6fd
    # +?は直前のパターンを1回以上繰り返す、という意味なので、"\w+?"で「1文字以上の英数字」を表す。
    # +は直前のパターンを1回以上繰り返す、という意味なので、"([0-9]+)"は「1文字以上の数字」を表す。また、([0-9]+)の()はキャプチャグループ化を意味する。
    # "\1"は1番目のキャプチャグループを意味する。つまり、ココでは([0-9]+)になる。https://chat.openai.com/share/e08ae899-4426-48df-aaf6-bdf78e87fd9e

# 5万円 → 5
def drop_man(x):
    return re.sub(r'([0-9]+)万円+',r'\1',x)
    # 万円を"\w+"にすると、31.2万円 → 3.2。32.2 → 3.2になる。なぜ？

# 数字+文字 → 文字
def numobj_num(x):
    return re.sub(r'([0-9]+)\w+',r'\1',x)
# ([0-9]+)\w+は、1文字以上の数字＋1文字以上の英数字。"\1"は1番目のキャプチャグループを返す。

# 英字以降を切り落とし
def drop_behind_alfa(x):
    return re.sub('[a-z]\w+','',x)

# 文字列の検索。返り値はTrue,False
def search_object(search, object):
    return bool(re.search(search,object))
# re.search(検索パターン, 検索対象)で該当すれば、matchしたオブジェクトを返す。https://note.nkmk.me/python-re-match-search-findall-etc/#search
# ここでは、bool()を用いているので、re.search()でマッチすればTrue、NoneであればFalseを返す。

# 最後の数字列を取りだす
def lastnum(x):
    nums = re.findall('[0-9]+', x)
    
    if nums:
        return nums[-1]
    else:
        # 数字が見つからない場合の処理。0を返す。
        return 0

# 文字+数字 → 数字
def get_objnum(x):
    return re.search(r'([A-Z]?)([0-9]+)',x).group()  # re.search()でマッチしたオブジェクトに対して、.group()でオブジェクトのテキストデータのみ出力

# 「地下」という文字列が入っていればその階数、入っていなければ0
def underground(x):
    return search_object('地下',x)*objnumobj_num(x)+(1-search_object('地下',x))*'0'

def list_min(x):
    lists = re.findall(r'\w?\d+', x)
    converted_list = []
    
    for item in lists:
        if item.startswith('B'):
            converted_item = int(item.replace('B', '-'))
        else:
            converted_item = int(item)
        
        converted_list.append(converted_item)
        
    return min(converted_list)

In [35]:

data_list = []

# suumoサイトで東京都港区の賃貸物件を検索
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13103&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'
# pc=○○の所で一度に表示する件数を設定。pc=30では約20min、pc=50では約8min。

# スクレイピングページ数の指定
max_page = 200

# 1-指定ページまでをスクレイピング
for i in range(1, max_page + 1):

    target_url = url.format(i)
    res = requests.get(target_url)
    res.encoding = 'utf-8'

    # スクレイピング先のサーバー負荷軽減の為、1ページ情報を取得後に1秒のディレイ
    sleep(1)

    soup = BeautifulSoup(res.text, 'html.parser')

    # 物件情報の取得
    contents = soup.find_all('div',class_= 'cassetteitem')

    # 1物件ずつ情報の取得
    for content in contents:
        
        # 物件名、住所、アクセス、築年数、階建 情報ブロック
        detail = content.find('div',class_='cassetteitem-detail')
        # 階数、賃料、管理費、敷金、礼金、間取り、面積 情報ブロック
        table = content.find('table', class_='cassetteitem_other')

        # 物件名、住所、アクセス、築年数、階建の取得
        title = content.find('div', class_= 'cassetteitem_content-title').text
        address = content.find('li', class_= 'cassetteitem_detail-col1').text
        access_1 = content.find('li', class_= 'cassetteitem_detail-col2').find_all()[0].text
        access_2 = content.find('li', class_= 'cassetteitem_detail-col2').find_all()[1].text
        access_3 = content.find('li', class_= 'cassetteitem_detail-col2').find_all()[2].text
        age = content.find('li', class_= 'cassetteitem_detail-col3').find_all('div')[0].text
        story = content.find('li', class_= 'cassetteitem_detail-col3').find_all('div')[1].text

        # テーブルの行情報の取得
        tr_tags = table.find_all('tr',class_='js-cassette_link')

        # リスト1列毎に階数、賃料、管理費、敷金、礼金、間取り、面積の取得
        for tr_tag in tr_tags:
            floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]

            fee, management_fee = price.find_all('li')
            deposit, gratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')

            # 取得した情報を辞書に格納
            data = {
                'title':title,
                'address':address,
                'access_1':access_1,
                'access_2':access_2,
                'access_3':access_3,
                'age':age,
                'story':story,
                'floor':floor.text,
                'fee':fee.text,
                'management_fee':management_fee.text,
                'deposit':deposit.text,
                'gratuity':gratuity.text,
                'madori':madori.text,
                'menseki':menseki.text
            }

            data_list.append(data)

# データフレームを作成する
df = pd.DataFrame(data_list)

# カラム名の変更
df.columns = ['物件名', '住所', 'アクセス1', 'アクセス2', 'アクセス3', '築年数', '階建', '階数', '賃料', '管理費', '敷金', '礼金', '間取り', '面積']

In [36]:
# データのクレンジング

# 築年数
df.loc[df['築年数']=='新築','築年数'] = '築0年'                              # 新築は築0年。
                                                                            # df.loc[df['築年数']=='新築']は、'築年数'カラムの値が'新築'のものを抽出。
                                                                            # df.loc[df['築年数']=='新築','築年数']は抽出したデータフレームから、'築年数'カラムのデータを抽出したもの。ここでは、シリーズ形式。['築年数']であればデータフレーム形式。https://qiita.com/Tusnori/items/31746dd1c55ecff2bb22
df['築年数'] = df['築年数'].map(lambda x: objnumobj_num(x)).astype(int)     # 築5年 → 5、int型に変更

In [37]:
# 階建
df.loc[df['階数']=='平屋', '階数'] = '1階建'
df['地下'] = df['階建'].map(lambda x: underground(x)).astype(int)
df['地上'] = df['階建'].map(lambda x: lastnum(x)).astype(int)
df['階建'] = df['地上'] + df['地下']

In [38]:
# 階数
df['階数'] = df['階数'].str.strip()
df.loc[df['階数']=='-','階数'] = '1階'
df['階数'] = df['階数'].map(lambda x: numobj_num(x))
df['階数'] = df['階数'].map(lambda x: list_min(x))

In [39]:
# 賃料
df['賃料'] = df['賃料'].map(lambda x: drop_man(x)).astype(float)            # 5万 → 5、float型に変更

In [40]:
# 管理費
df.loc[df['管理費']=='-','管理費'] = '0円'                                  # '-' → '0円'
df['管理費'] = df['管理費'].map(lambda x:numobj_num(x)).astype(int)         # 5円 → 5、int型に変更

In [41]:
# 敷金
df.loc[df['敷金']=='-','敷金'] = '0万円'                                    # '-' → '0万円'
df['敷金'] = df['敷金'].map(lambda x: drop_man(x)).astype(float)            # 5万円 → 5、float型に変更

In [42]:
# 礼金
df.loc[df['礼金']=='-','礼金'] = '0万円'                                    # '-' → '0万円'
df['礼金'] = df['礼金'].map(lambda x: drop_man(x)).astype(float)            # 5万円 → 5、float型に変更

In [43]:
# 間取り
df.loc[df['間取り']=='ワンルーム','間取り'] = '1'                            # 'ワンルーム' → '1'   間取りのSはサービスルーム
df['間取り_部屋数'] = df['間取り'].map(lambda x:numobj_num(x)).astype(int)

In [44]:
# 面積
df['面積'] = df['面積'].map(lambda x: drop_behind_alfa(x)).astype(float)    # 5m2 → 5

In [58]:
df = df.sort_values('物件名')

In [59]:
# df['階数'].sort_values().unique()

In [60]:
df.head()
df1 = df.drop_duplicates(subset=['物件名'                                                                                   ],keep='first')
df2 = df.drop_duplicates(subset=['物件名', '住所'                                                                           ],keep='first')
df3 = df.drop_duplicates(subset=['物件名', '住所',                          '賃料', '管理費', '敷金', '礼金', '間取り', '面積'],keep='first')
df4 = df.drop_duplicates(subset=['物件名', '住所', '築年数', '階建', '階数', '賃料', '管理費', '敷金', '礼金', '間取り', '面積'],keep='first')
df5 = df.drop_duplicates(subset=[                  '築年数', '階建', '階数', '賃料', '管理費', '敷金', '礼金', '間取り', '面積'],keep='first')

In [61]:
print('df :',len(df))
print('df1:',len(df1))
print('df2:',len(df2))
print('df3:',len(df3))
print('df4:',len(df4))
print('df5:',len(df5))

df : 9625
df1: 3381
df2: 3504
df3: 8646
df4: 8916
df5: 6447


In [62]:
# dfから値を習得
values = [df5.columns.values.tolist()] + df5.values.tolist()

# スプレッドシートにデータを追加
worksheet.update("A1", values)

C:\Users\yuki_yamakawa\AppData\Local\Temp\ipykernel_45956\1753189510.py:5: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("A1", values)


{'spreadsheetId': '1RN3MBSM-864r1Me1hdgOZmWzXY0NLMJsb7O6FBogg6k',
 'updatedRange': "'シート1'!A1:Q6448",
 'updatedRows': 6448,
 'updatedColumns': 17,
 'updatedCells': 109616}